In diesem Notebook wird der Datensatz base_fake_real.csv verwendet und untersucht wie man Fake-Reviews von Echten Reviews unterscheiden kann.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib
import os

In [ ]:
from fastai.vision.all import *
from fastdownload import download_url

## Text:

Über Sprach-Features aus PNLP

## Bild:
Erkennung, ob das Bild ein Fake ist oder nicht über eine Bildklassifizierung.

In [ ]:
df = pd.read_csv('base_keywords_sentiment_reduced.csv')
print(df.shape)

df_all = pd.read_csv('01_Data/raw_data/dataset_weitere_forschung.csv')
print(df_all.shape)

df_all = df_all[['text', 'reviewId', 'url', 'placeId', 'categoryName', 'stars', 'title','reviewImageUrls/0','reviewImageUrls/1','reviewImageUrls/2','reviewImageUrls/3','reviewImageUrls/4','reviewImageUrls/5','genre']]
df_all = df_all[df_all['reviewImageUrls/0'].notna()]
print(df_all.shape)
df_all = df_all[~df_all['reviewId'].isin(df['reviewId'])]
print(df_all.shape)
print(df_all.columns)

image_urls_cols = [
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
]
for col in image_urls_cols:
    print(f"Processing col {col}")
    df_all[col] = df_all[col].str.replace("=w150-h150-k-no-p", "=w256-h256-p-k-no")

print(df_all.shape)

# df_all.to_csv('train_weitere_forschung_ohne_base.csv', index=False)
# df_all.to_excel('train_weitere_forschung_ohne_base.xlsx', index=False)

In [ ]:
df = pd.read_csv('train_weitere_forschung_ohne_base.csv')

df = df.sample(n=20, random_state=8)
# df = df.sample(n=1000, random_state=42)
df = df.sort_index()

for index, row in df.iterrows():
    if index < 0:
        print(f"Index {index} schon heruntergeladen, skipped ..")
        continue
    for i in range(2):
        if not pd.isna(row[f'reviewImageUrls/{i}']):
            print(f"Downloading image for {index}...")
            url = row[f'reviewImageUrls/{i}']
            filename = f"{url.split('/')[-1]}.png"
            path = os.path.join('02_Images', 'train', 'real', filename)
            if not os.path.exists(path):
                try:
                    urllib.request.urlretrieve(url, path)
                except:
                    print(f'Error with {url}')

print("Done with downloading real images for training set.")

In [ ]:
# dwonlad for every row the file of column 

### Start Umsetzung mit FASTAI visual Learning als Classifizierung Problem (AUF COLAB!)

In [ ]:
path = Path('02_Images/train')
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
print(failed)
if len(failed) > 0:
    failed.map(Path.unlink);
else:
    print("All images verified successfully.")

In [ ]:
classes = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(256))

print("Done with creating DataBlock.")

In [ ]:
dls = classes.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)
print(len(dls.valid.items))

In [ ]:
classes = classes.new(item_tfms=Resize(256, ResizeMethod.Squish))
dls = classes.dataloaders(path)

classes = classes.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = classes.dataloaders(path)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(epochs = 4, freeze_epochs=1)

Auswertung des Image-Klassifikators

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

interp.plot_confusion_matrix(figsize=(12,12), dpi=60)
plt.savefig('image_classificator_confusion_matrix.svg', format='svg', dpi=1200)

In [ ]:
print("Die Bilder mit den schlechtesten loss Werte.")
interp.plot_top_losses(25, nrows=10)

## Anwendung des Image Classificators auf die url0

In [28]:
from fastai.vision.all import load_learner
import pathlib
import os

In [29]:
learn_inf = load_learner('img_classif_trained_reviewUrl12.pkl')
prediction = learn_inf.predict('path/to/image.jpg')

print(prediction)


NotImplementedError: cannot instantiate 'PosixPath' on your system

In [ ]:
# Get the absolute path to the pkl file
pkl_file = pathlib.Path('img_classificator_train_url12.pkl').resolve()

# Load the pkl file
learn_inf = load_learner(pkl_file)

# Use the learner object for prediction
prediction = learn_inf.predict('path/to/image.jpg')

print(prediction)


In [ ]:
# read the data from base_fake_real.csv

df = pd.read_csv('base_fake_real.csv')
print(df.shape)

# add a new column 'prediced_img_clsf' with the predicted class of the image classifier
df['prediced_img_clsf'] = np.nan

# load a pkl model with the trained model for FASTAI




## Tabulare Daten:



## Erst Text, dann Bildverifizierung